In [1]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader

# Import de tes modules custom
sys.path.append("..")
from src.data_utils import BrainScanDataset
from src.models import get_brainscan_model
from src.engine import train_one_epoch, evaluate_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Moteur de calcul : {device}")

🚀 Moteur de calcul : cpu


### Stratégie de split

In [2]:
df = pd.read_csv("../data/metadata_with_weak_labels.csv")

# 1. Isolation du Test Set (Uniquement des labels forts, jamais vus)
df_strong_all = df[df["label"] != -1].copy()
df_test = df_strong_all.sample(n=20, random_state=42)
df_train_strong = df_strong_all.drop(df_test.index)

# 2. Le set de Training Faible (les 1406 images issues du clustering)
df_train_weak = df[df["label"] == -1].copy()

# Sauvegarde temporaire pour les DataLoaders
df_test.to_csv("../data/temp_test.csv", index=False)
df_train_strong.to_csv("../data/temp_train_strong.csv", index=False)
df_train_weak.to_csv("../data/temp_train_weak.csv", index=False)

### Préparation des DataLoaders

In [3]:
from torchvision import transforms

# Transformation standard (celle de l'étape 2)
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# DataLoaders
# Note : Pour train_weak, on utilise 'weak_label' généré à l'étape 3
loader_weak = DataLoader(
    BrainScanDataset(
        "../data/temp_train_weak.csv", transform=train_transform, label_col="weak_label"
    ),
    batch_size=32,
    shuffle=True,
)
loader_strong = DataLoader(
    BrainScanDataset(
        "../data/temp_train_strong.csv", transform=train_transform, label_col="label"
    ),
    batch_size=16,
    shuffle=True,
)
loader_test = DataLoader(
    BrainScanDataset(
        "../data/temp_test.csv", transform=train_transform, label_col="label"
    ),
    batch_size=16,
    shuffle=False,
)

### Expérience A - Baseline Supervisée
#### Le modèle apprend uniquement sur les 80 images de Clara.

In [4]:
print("--- EXPÉRIENCE A : SUPERVISÉ PUR ---")
model_sup = get_brainscan_model(device=device)
optimizer = optim.Adam(model_sup.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    loss = train_one_epoch(model_sup, loader_strong, optimizer, criterion, device)
    if epoch % 2 == 0:
        print(f"Époque {epoch} | Loss: {loss:.4f}")

f1_sup, report_sup = evaluate_model(model_sup, loader_test, device)

--- EXPÉRIENCE A : SUPERVISÉ PUR ---


Époque 0 | Loss: 0.4481


Époque 2 | Loss: 0.0480


Époque 4 | Loss: 0.0084


Époque 6 | Loss: 0.0019


Époque 8 | Loss: 0.0044


### Expérience B - Semi-Supervisée (Pseudo-Labeling)
#### Phase 1 sur les labels "faibles", puis Phase 2 sur les labels "forts".

In [5]:
# --- EXPÉRIENCE B : SEMI-SUPERVISÉ (AVEC LOGGING DES ÉPOQUES) ---
print("--- EXPÉRIENCE B : SEMI-SUPERVISÉ ---")

# 1. Alignement automatique des labels
mapping_counts = pd.crosstab(df_strong_all["label"], df["weak_label"])
correct_label_for_cancer = int(mapping_counts.loc[1].idxmax())

if correct_label_for_cancer == 0:
    print("⚠️ Inversion détectée ! Correction des weak_labels en cours...")
    df["weak_label"] = df["weak_label"].map({0: 1, 1: 0})
else:
    print("✅ Alignement des labels correct.")

# 2. Préparation des loaders
df_train_weak = df[df["label"] == -1].copy()
loader_weak = DataLoader(
    BrainScanDataset(df_train_weak, transform=train_transform, label_col="weak_label"),
    batch_size=32,
    shuffle=True,
)

# 3. Initialisation
model_semi = get_brainscan_model(num_classes=2, device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_semi.parameters(), lr=1e-4)

# PHASE 1 : Pré-entraînement (Labels Faibles)
num_epochs_p1 = 5
print(f"\n🚀 Phase 1 : Pre-training ({num_epochs_p1} époques)")
for epoch in range(num_epochs_p1):
    loss = train_one_epoch(model_semi, loader_weak, optimizer, criterion, device)
    # Affichage clair : Epoch 1/5, Epoch 2/5, etc.
    print(f" > Époque [{epoch+1}/{num_epochs_p1}] | Loss Moyenne: {loss:.4f}")

# PHASE 2 : Fine-tuning (Labels Forts)
num_epochs_p2 = 5
print(f"\n🚀 Phase 2 : Fine-tuning ({num_epochs_p2} époques)")
optimizer = optim.Adam(model_semi.parameters(), lr=1e-5)

for epoch in range(num_epochs_p2):
    loss = train_one_epoch(model_semi, loader_strong, optimizer, criterion, device)
    print(f" > Époque [{epoch+1}/{num_epochs_p2}] | Loss Moyenne: {loss:.4f}")

# 4. Évaluation
f1_semi, report_semi = evaluate_model(model_semi, loader_test, device)
print(f"\n📊 Résultat Final Semi-Supervisé : F1 = {f1_semi:.4f}")
print(report_semi)

--- EXPÉRIENCE B : SEMI-SUPERVISÉ ---
⚠️ Inversion détectée ! Correction des weak_labels en cours...

🚀 Phase 1 : Pre-training (5 époques)


 > Époque [1/5] | Loss Moyenne: 0.1137


 > Époque [2/5] | Loss Moyenne: 0.0211


 > Époque [3/5] | Loss Moyenne: 0.0205


 > Époque [4/5] | Loss Moyenne: 0.0214


...

 > Époque [1/5] | Loss Moyenne: 0.9270


 > Époque [2/5] | Loss Moyenne: 0.5634


 > Époque [3/5] | Loss Moyenne: 0.3067


 > Époque [4/5] | Loss Moyenne: 0.1922


 > Époque [5/5] | Loss Moyenne: 0.1193

📊 Résultat Final Semi-Supervisé : F1 = 0.8571
              precision    recall  f1-score   support

      Normal       0.86      1.00      0.92        12
      Cancer       1.00      0.75      0.86         8

    accuracy                           0.90        20
   macro avg       0.93      0.88      0.89        20
weighted avg       0.91      0.90      0.90        20



### Comparaison des résultats

In [6]:
print(f"📊 F1-Score Supervisé : {f1_sup:.4f}")
print(f"📊 F1-Score Semi-Supervisé : {f1_semi:.4f}")

if f1_semi > f1_sup:
    print("\n✅ Succès : L'approche semi-supervisée améliore la généralisation.")
else:
    print(
        "\n⚠️ Note : Le gain est faible, la qualité des clusters de l'étape 3 est à revoir."
    )

📊 F1-Score Supervisé : 1.0000
📊 F1-Score Semi-Supervisé : 0.8571

⚠️ Note : Le gain est faible, la qualité des clusters de l'étape 3 est à revoir.


In [7]:
# Vérification du mapping
print(pd.crosstab(df_strong_all["label"], df_strong_all["weak_label"]))

weak_label   0   1
label             
0            2  48
1           38  12
